<a href="https://colab.research.google.com/github/victor7543/TechChallenge-Reconhecimento-Facial/blob/master/Tech4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface
!pip install insightface
!pip install mediapipe
!pip install opencv-python
!pip install onnxruntime-gpu

In [ ]:
!pip install insightface deepface mediapipe opencv-python-headless


In [ ]:
!wget -O hand_landmarker.task \
https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task


--2026-01-06 23:58:02--  https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.207, 108.177.98.207, 74.125.135.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7819105 (7.5M) [application/octet-stream]
Saving to: ‘hand_landmarker.task’

hand_landmarker.tas 100%[===================>]   7.46M  --.-KB/s    in 0.02s   

2026-01-06 23:58:02 (311 MB/s) - ‘hand_landmarker.task’ saved [7819105/7819105]



In [ ]:
!ls -lh hand_landmarker.task


-rw-r--r-- 1 root root 7.5M Apr 26  2023 hand_landmarker.task


In [ ]:
import cv2
import numpy as np
import os
from deepface import DeepFace
from insightface.app import FaceAnalysis
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ==================================================
# 1. CONFIGURAÇÕES PRINCIPAIS
# ==================================================
VIDEO_PATH = '/content/IMG_7753.mov'
OUTPUT_VIDEO_PATH = 'video_final_analise.mp4'

FRAME_SKIP = 1
ANOMALY_THRESHOLD_MOVE = 50
ANOMALY_THRESHOLD_EMOTION = 2
ANOMALY_CONFIDENCE_THRESHOLD = 15
ANOMALY_EMOTIONS = ['surprise', 'angry', 'disgust', 'sad', 'fear']
SIMILARITY_THRESHOLD = 0.55

# ==================================================
# 2. INSIGHTFACE
# ==================================================
app = FaceAnalysis(
    name='buffalo_l',
    allowed_modules=['detection', 'recognition'],
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
)
app.prepare(ctx_id=0, det_size=(640, 640))

# ==================================================
# 3. MEDIAPIPE HAND LANDMARKER (MODERNO)
# ==================================================
HAND_CONNECTIONS = [
    (0,1),(1,2),(2,3),(3,4),
    (0,5),(5,6),(6,7),(7,8),
    (0,9),(9,10),(10,11),(11,12),
    (0,13),(13,14),(14,15),(15,16),
    (0,17),(17,18),(18,19),(19,20)
]

def draw_hand_landmarks(frame, hand_landmarks, color=(0, 255, 255)):
    h, w, _ = frame.shape
    pts = []
    for lm in hand_landmarks:
        x = int(lm.x * w)
        y = int(lm.y * h)
        pts.append((x, y))
        cv2.circle(frame, (x, y), 3, color, -1)
    for s, e in HAND_CONNECTIONS:
        cv2.line(frame, pts[s], pts[e], color, 2)

base_options = python.BaseOptions(model_asset_path="hand_landmarker.task")
hand_options = vision.HandLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_hands=2
)
hand_landmarker = vision.HandLandmarker.create_from_options(hand_options)

# ==================================================
# 4. VARIÁVEIS DE CONTROLE
# ==================================================
frame_count = 0
analyzed_frame_count = 0
total_face_detections = 0
anomalies_count = 0
previous_face_position = {}
previous_emotion = {}
unique_faces_embeddings = []
unique_face_id_counter = 0

emotion_tally = {
    'angry': 0, 'disgust': 0, 'fear': 0,
    'happy': 0, 'sad': 0, 'surprise': 0, 'neutral': 0
}
anomaly_emotion_count = 0

print("Iniciando análise COMPLETA de 100% dos frames.")

# ==================================================
# 5. ABERTURA DO VÍDEO
# ==================================================
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    VIDEO_PATH = '/content/' + VIDEO_PATH
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        os.system(f"touch {OUTPUT_VIDEO_PATH}")
        exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

# ==================================================
# 6. LOOP PRINCIPAL
# ==================================================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    analyzed_frame_count += 1

    # ---------- FACES ----------
    faces_detected = app.get(frame)

    # ---------- HANDS ----------
    timestamp_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)
    hand_result = hand_landmarker.detect_for_video(mp_image, timestamp_ms)

    hands_detected = bool(hand_result.hand_landmarks)
    num_hands = len(hand_result.hand_landmarks) if hands_detected else 0

    if hand_result.hand_landmarks:
        for hand_landmarks in hand_result.hand_landmarks:
            draw_hand_landmarks(frame, hand_landmarks)

    try:
        for face_info in faces_detected:
            bbox = face_info.bbox.astype(np.int32)
            x, y, x2, y2 = bbox
            face_crop = frame[y:y2, x:x2]

            analysis = DeepFace.analyze(
                img_path=face_crop,
                actions=['emotion'],
                enforce_detection=False
            )
            if not analysis:
                continue

            dominant_emotion = analysis[0]['dominant_emotion']
            emotion_scores = analysis[0]['emotion']

            total_face_detections += 1
            emotion_tally[dominant_emotion] += 1

            embedding = face_info.embedding
            detected_id = -1

            for i, stored_emb in enumerate(unique_faces_embeddings):
                sim = np.dot(embedding, stored_emb) / (
                    np.linalg.norm(embedding) * np.linalg.norm(stored_emb)
                )
                if (1 - sim) < SIMILARITY_THRESHOLD:
                    detected_id = i
                    break

            if detected_id == -1:
                detected_id = unique_face_id_counter
                unique_faces_embeddings.append(embedding)
                unique_face_id_counter += 1

            face_id = detected_id

            activity_label = "Interacao Social"
            color = (0, 255, 0)

            cx, cy = (x + x2) // 2, (y + y2) // 2
            if face_id in previous_face_position:
                px, py = previous_face_position[face_id]
                dist = ((cx - px)**2 + (cy - py)**2)**0.5
                if dist > ANOMALY_THRESHOLD_MOVE:
                    anomalies_count += 1
                    activity_label = "**ANOMALIA (Mov. Brusco)**"
                    color = (0, 0, 255)

            previous_face_position[face_id] = (cx, cy)

            is_anomaly_expression = False
            anomaly_trigger = dominant_emotion

            if activity_label != "**ANOMALIA (Mov. Brusco)**":
                for emo in ANOMALY_EMOTIONS:
                    if emotion_scores.get(emo, 0) > ANOMALY_CONFIDENCE_THRESHOLD:
                        is_anomaly_expression = True
                        if emotion_scores[emo] > emotion_scores.get(anomaly_trigger, 0):
                            anomaly_trigger = emo

            prev_emo = previous_emotion.get(face_id, 'neutral')

            if is_anomaly_expression and prev_emo in ['neutral', 'happy']:
                anomaly_emotion_count += 1
                anomalies_count += 1
                activity_label = f"**ANOMALIA ({anomaly_trigger.capitalize()})**"
                color = (255, 0, 0)

            elif hands_detected:
                if num_hands >= 2:
                    activity_label = "Digitando/Usando Dispositivo"
                elif dominant_emotion in ['neutral', 'sad']:
                    activity_label = "Lendo/Focado"

            previous_emotion[face_id] = dominant_emotion

            cv2.rectangle(frame, (x, y), (x2, y2), color, 2)
            cv2.putText(frame, f"ID:{face_id} Emo:{dominant_emotion}",
                        (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            cv2.putText(frame, activity_label,
                        (x, y - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    except Exception:
        pass

    out.write(frame)

# ==================================================
# 7. FINALIZAÇÃO
# ==================================================
cap.release()
out.release()
cv2.destroyAllWindows()

print("\n" + "="*70)
print("✨ RELATÓRIO FINAL ✨")
print("="*70)
print(f"Vídeo salvo em: {OUTPUT_VIDEO_PATH}")
print(f"Faces detectadas: {total_face_detections}")
print(f"Pessoas únicas: {unique_face_id_counter}")
print(f"Frames analisados: {frame_count}")
print(f"Anomalias totais: {anomalies_count}")
print(f"Anomalias por expressão: {anomaly_emotion_count}")
print("="*70)


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvid

In [ ]:
from google.colab import files
files.download('video_final_analise.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>